In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch
import warnings
warnings.filterwarnings('ignore')

In [2]:
FILE_URL = '/home/rodrigo/Escritorio/reto06/frecuencia_palabras.csv'
response = pd.read_csv((FILE_URL), sep = ';')
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
response.dtypes

URL                  object
Python               object
Machine_learning     object
Analytics            object
Visualization        object
Clustering           object
Models               object
Pandas               object
Business             object
SQL                  object
dtype: object

In [4]:
es.indices.delete(index = 'index_interactions', ignore = [400, 404])

{'acknowledged': True}

In [5]:
df_interactions= response

In [6]:
df_interactions = df_interactions.reset_index()

In [7]:
df_interactions.head(8)

,index,URL,Python,Machine_learning,Analytics,Visualization,Clustering,Models,Pandas,Business,SQL
0,0,URL 1,1,0,4,2,0,0,1,0,1
1,1,URL 2,1,2,1,0,0,2,0,0,0
2,2,URL 3,1,0,3,0,0,0,0,0,1
3,3,URL 4,0,2,0,0,0,1,0,1,0
4,4,URL 5,2,0,0,2,0,1,0,0,2
5,5,URL 6,1,0,0,0,0,2,1,0,0
6,6,URL 7,1,0,0,0,0,0,0,0,1
7,7,Porcentaje,85.57%,28.57%,42.85%,28.57%,0.0%,57.14%,28.57%,14.28%,57.14%


In [8]:
df_interactions = df_interactions.head(7) # usaremos solamente el conteo de las skills en las diferentes URLs 

In [9]:
df_interactions

,index,URL,Python,Machine_learning,Analytics,Visualization,Clustering,Models,Pandas,Business,SQL
0,0,URL 1,1,0,4,2,0,0,1,0,1
1,1,URL 2,1,2,1,0,0,2,0,0,0
2,2,URL 3,1,0,3,0,0,0,0,0,1
3,3,URL 4,0,2,0,0,0,1,0,1,0
4,4,URL 5,2,0,0,2,0,1,0,0,2
5,5,URL 6,1,0,0,0,0,2,1,0,0
6,6,URL 7,1,0,0,0,0,0,0,0,1


In [10]:
index = list(df_interactions.index)

In [11]:
index

[0, 1, 2, 3, 4, 5, 6]

In [12]:
idd = 1
for i in index:
    
    doc = {
        
        'URL': df_interactions.loc[i,'URL'],
        'Python': df_interactions.loc[i,'Python'],
        'Analytics': df_interactions.loc[i,'Analytics'],
        'Visualization': df_interactions.loc[i,'Visualization'],
        'Clustering': df_interactions.loc[i,'Clustering'],
        'Models': df_interactions.loc[i,'Models'],
        'Pandas': df_interactions.loc[i,'Pandas'],
        'Business': df_interactions.loc[i,'Business'],
        'SQL': df_interactions.loc[i,'SQL'],
        'timestamp': datetime.now()
    }
    res = es.index(index = 'index_interactions', id = idd, document = doc)
    idd +=1

# Consultas

1. Casos en los que "Python" aparece en dos ocasiones en el mismo texto:

In [34]:
res = es.search(index = 'index_interactions', query = {"match": {'Python' : '2'}})

In [35]:
res['hits']['total']['value']

1

2. Devolver el índice de aquellas URL cuyo "SQL" sea mayor o igual a 1:

In [15]:
res = es.search(index="index_interactions", query={"range": {"SQL": {"gte":1}}})

In [16]:
res['hits']['total']['value']

0

In [17]:
for hit in res["hits"]["hits"]:
    print(hit["_source"])
    print("\n")

3. Casos en los que "Analytics" aparece en 3 ocasiones o más en el mismo texto:

In [18]:
res = es.search(index="index_interactions", query={"range": {"Analytics": {"gte":3}}})

In [19]:
res['hits']['total']['value']

2

In [20]:
for hit in res["hits"]["hits"]:
    print(hit["_source"]),          # Vemos como de los 7 textos, solo 3 contienen la palabra, y de esos 3
    print("\n")                     # unicamente en 2 aparece más de 3 veces.

{'URL': 'URL 1', 'Python': '1', 'Analytics': '4', 'Visualization': '2', 'Clustering': '0', 'Models': '0', 'Pandas': '1', 'Business': '0', 'SQL': '1', 'timestamp': '2022-01-16T17:35:09.058618'}


{'URL': 'URL 3 ', 'Python': '1', 'Analytics': '3', 'Visualization': '0', 'Clustering': '0', 'Models': '0', 'Pandas': '0', 'Business': '0', 'SQL': '1', 'timestamp': '2022-01-16T17:35:09.747296'}




4. Casos en los que "Clustering" aparece al menos en una ocasion en el texto:

In [21]:
res = es.search(index = 'index_interactions', query = {"match": {'Clustering' : '1'}})

In [22]:
res['hits']['total']['value']          # La palabra "Clustering" no ha aparecido en ni un solo texto.

0

5. Casos en los que "Models" aparece al menos dos veces en un texto:

In [23]:
res = es.search(index = 'index_interactions', query = {"match": {'Models' : '2'}})

In [24]:
res['hits']['total']['value']

2

In [25]:
for hit in res["hits"]["hits"]:
    print(hit["_source"]),          # Vemos como de los 7 textos, 4 contienen la palabra, y de esos 4
    print("\n")                     # unicamente en 2 aparece más de 2 veces.

{'URL': 'URL 2', 'Python': '1', 'Analytics': '1', 'Visualization': '0', 'Clustering': '0', 'Models': '2', 'Pandas': '0', 'Business': '0', 'SQL': '0', 'timestamp': '2022-01-16T17:35:09.694995'}


{'URL': 'URL 6', 'Python': '1', 'Analytics': '0', 'Visualization': '0', 'Clustering': '0', 'Models': '2', 'Pandas': '1', 'Business': '0', 'SQL': '0', 'timestamp': '2022-01-16T17:35:09.826975'}




6. Casos en los que "Visualization" aparece en una ocasion en el mismo texto:

In [39]:
res = es.search(index = 'index_interactions', query = {"match": {'Visualization' : '1'}})

In [40]:
res['hits']['total']['value']

0

In [41]:
for hit in res["hits"]["hits"]:
    print(hit["_source"]) # En las dos ocasiones en las que aparecen las palabras "Visualization", aparece por 
    print("\n")           # lo menos dos veces, por lo que nos devuelve 0. Lo comprobamos en la siguiente consulta.

7. Casos en los que "Visualization" aparece en dos ocasiones en el mismo texto:

In [42]:
res = es.search(index = 'index_interactions', query = {"match": {'Visualization' : '2'}})

In [43]:
res['hits']['total']['value']

2

In [44]:
for hit in res["hits"]["hits"]:
    print(hit["_source"])
    print("\n")

{'URL': 'URL 1', 'Python': '1', 'Analytics': '4', 'Visualization': '2', 'Clustering': '0', 'Models': '0', 'Pandas': '1', 'Business': '0', 'SQL': '1', 'timestamp': '2022-01-16T17:35:09.058618'}


{'URL': 'URL 5 ', 'Python': '2', 'Analytics': '0', 'Visualization': '2', 'Clustering': '0', 'Models': '1', 'Pandas': '0', 'Business': '0', 'SQL': '2', 'timestamp': '2022-01-16T17:35:09.807104'}


